<h1>Washington State Hunting Statistics</h1>
<h2>Eric Lagally</h2>
<h2>eric.lagally@gmail.com</h2>
<h3>April 2024</h3>

This script scrapes the Washington State Department of Fish and Wildlife (WDFW) website for public data on hunting outcomes as a function of state region, species, weapon, year, and more.  It collects these data into a dataframe and then exports that as an Excel file with various tabs organized into a relational format that is amenable to analysis using Tableau Public.  


In [ ]:
import requests
import re
import os
import pandas as pd
from bs4 import BeautifulSoup
import openpyxl


In [ ]:
columns = ["No antlers","Antlered total","Total","1 Point","2 Point","3 point","4 point","5+ point","Number Hunters","Hunter Success","Hunter Days","Days/Kill","GMU","Weapon","Year"]
elkcols = ["No antlers","Antlered total","Total","1 Point","2 Point","3 point","4 point","5 point","6+ points","Number Hunters","Hunter Success","Hunter Days","Days/Kill","GMU","Weapon","Year"]
shortcols = ["GMU","No antlers","Antlered total","Total","1 Point","2 Point","3 point","4 point","5+ point","Year"]
elk_shortcols = ["GMU","No antlers","Antlered total","Total","1 Point","2 Point","3 point","4 point","5 point","6+ point", "Year"]
bearcols = ["GMU","Male","Female","Total","Number Hunters","Hunter Success","Hunter Days","Days/Kill","Year"]
bearspr = ["GMU","Applicants","Permits","Reports","Males","Females","Total","Hunters","Hunter Success","Year"]
turkeycols = ["Season","PMU","PMU Name","Total Kills","Number Hunters","Hunter Success","Hunter Days","Days/Kill","Year"]
smallgamecols = ["Species","County","Total Kills","Hunters","Days","Year"]
trappingcols = ["County","Badger","Beaver","Coyote","Mink","Muskrat","Nutria","Otter","Raccoon","Skunk","Weasel","Other","Year"]
bighorn_cols = ["Unit","Applicants","Quota","Permits Issued","Reports","Kills","Hunters","Hunter Days","Days/Kill","Hunter Success","Year"]
moosecols = ["Unit","Applicants","Quota","Permits Issued","Reports","Kills","Males","Females","Hunters","Hunter Days","Days/Kill","Hunter Success","Year"]
mountaingoat_cols = ["Unit","Applicants","Quota","Permits Issued","Reports","Kills","Males","Females","Hunters","Hunter Days","Days/Kill","Hunter Success","Year"]
cougarcols = ["GMU","Male General","Female General","Unknown General","Total General","Depredation Male","Depredation Female","Depredation Unknown","Depredation Total","Other Male","Other Female","Other Unknown","Unknown Total","Total","Year"]

year_list = ["2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]
gmu = {"GMU":[101,105,108,111,113,117,121,124,127,130,133,136,139,142,145,149,154,157,162,163,166,169,172,175,178,181,186,203,204,209,215,218,224,
              231,233,239,242,243,244,245,246,247,248,249,250,251,254,260,262,266,269,272,278,284,290,328,329,330,334,335,336,340,342,346,352,
              356,360,364,368,371,372,373,379,381,407,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,426,437,448,450,454,460,466,
              485,490,382,388,501,503,504,505,506,510,513,516,520,522,524,530,550,554,556,560,564,568,572,574,578,601,602,603,607,612,615,618,
              621,624,627,633,636,638,642,648,651,652,653,654,655,658,660,663,666,667,672,673,681,684,699],
       "PMU":["P10","P10","P10","P10","P10","P10","P10","P10","P10","P10","P10","P10","P15","P15","P15","P15","P15","P15","P15","P15","P15","P15",
              "P15","P15","P15","P15","P15","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20",
              "P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P20","P30","P30","P30","P30","P30","P30","P30","P30","P30","P30","P30",
              "P30","P30","P30","P30","P30","P30","P30","P30","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40",
              "P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40","P35","P35","P50","P50","P50","P50","P50","P50","P50","P50","P50",
              "P50","P50","P50","P50","P50","P50","P50","P50","P35","P35","P35","P35","P40","P40","P40","P40","P40","P40","P40","P40","P40","P40",
              "P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50","P50"],
       "BMU":[7,7,7,7,7,7,7,7,7,7,7,9,9,9,8,8,8,None,8,8,8,8,8,8,8,8,8,5,7,5,5,5,5,5,5,5,5,5,6,6,6,6,9,6,6,6,9,9,9,9,9,9,None,9,9,6,6,None,6,6,6,
              6,6,6,6,6,6,6,6,None,9,9,None,9,2,None,None,None,None,None,None,None,None,3,None,None,None,None,None,None,3,3,3,3,2,3,4,4,None,6,6,
              1,4,1,4,1,4,4,4,4,None,4,1,4,4,4,4,4,4,4,4,6,1,1,1,1,1,1,1,1,2,2,2,1,1,1,1,1,2,4,4,None,1,1,1,2,2,1,1,1,1,1],
       "District":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,
                   3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,
                   6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6]
      }
mooseLUT = pd.read_csv("mooseLUT.csv")
bighornLUT = pd.read_csv("BighornLUT.csv")
MGLUT = pd.read_csv("MountainGoatLUT.csv")

In [ ]:
def deerelk(species: str, hunttype: str, year: str) -> pd.DataFrame:
    """
    deerelk is a function that scrapes the WDFW deer and elk hunting statistics web pages
    to extract information about the number of animals killed given a year, species, and 
    type of hunt (general season vs. special hunt) by location.

    Args:
        species: A string indicating either "deer" or "elk"
        hunttype:  A string indicating either "general" or "special"
        year: a string with a four digit year between 2013 and 2023

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    if hunttype=="general":
        if species == "deer":
            df = pd.DataFrame(columns=columns)
        else:
            df = pd.DataFrame(columns=elkcols)
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/"+species+"-general")
        page_html = response.text
        #print(page_html)
        soup = BeautifulSoup(page_html, 'lxml')
        counts = []
        districts = soup.find_all(class_="accordion-content")
        if year not in ["2013","2014","2015","2016","2017"]:
            for district in districts:
                tables = district.find_all("table")
                for table in tables:
                    gmu = table.find("caption")
                    name = gmu.contents[0].string
                    if re.search("Totals",name):
                        # ignore Totals sections
                        pass
                    else:
                        name = name[0:3]
                        body = table.find("tbody")
                        rows = body.find_all("tr")
                        for row in rows:
                            weapon = row.find(style="text-align: left;").string
                            data = row.find_all(style="text-align: right;")
                            for point in data:
                                counts.append(point.string)
                            counts.append(name)
                            # add the weapon type to the end of the list
                            counts.append(weapon)
                            counts.append(year)
                            # and add that list as a row in the data frame
                            df.loc[len(df)] = counts    
                            counts = []    
        else:
            for district in districts:
         
                gmus = district.find_all("h4")
                if not gmus:
                    # if page does not use h4 tags it uses h3 tags
                    gmus = district.find_all("h3")
                # for each GMU:
                for gmu in gmus:
                    name = gmu.contents[0].string
                    #print(name)
                    if re.search("Totals",name):
                        # ignore Totals sections
                        pass
                    else:
                        # grab just the GMU number
                        name = name[0:3]
                    
                        # find the tables in that GMU
                        table = gmu.next_element.next_element.next_element
                        #print(table)
                        body = table.find("tbody")
                        if body:
                            rows = body.find_all("tr")
                            for row in rows[1:]:
                                # is this a newer table format that has the text-align tag?
                                #print(row)
                                newweapon = row.find(style="text-align: left;")
                                if not newweapon:
                                    # nope, use the scope tag to set weapon type
                                    if not row.find(scope="row"):
                                        # ignore the silly empty row at the end of the older format tables
                                        pass
                                    else:
                                        weapon = row.find(scope="row").string
                                        #print(weapon)
                                else:
                                    # yes, newer table format, pull out the weapon type
                                    weapon = row.find(style="text-align: left;").string
                                # find all data points for that row
                                data = row.find_all(style="text-align: right;")
                                #print("Data is: ",data)
                            # is the table row data using the newer text-align tag?
                                if data:
                                    for point in data:
                                        # add the data from that row to a list
                                        counts.append(point.string)
                                    # add the GMU ID to the end of the list
                                    counts.append(name)
                                    # add the weapon type to the end of the list
                                    counts.append(weapon)
                                    counts.append(year)
                                    # and add that list as a row in the data frame
                                    df.loc[len(df)] = counts    
                                    counts = []
                                else:
                                    # nope, use the older table format
                                    if not row.find(scope="row"):
                                        # ignore the silly empty row at the end of the older format tables
                                        pass
                                    else:
                                        weapon = row.find(scope="row").string
                                        #print("no data weapon")
                                        data = row.find_all("td")
                                        #print(data)
                                        for point in data:
                                            counts.append(point.string)
                                        counts.append(name)
                                        counts.append(weapon)
                                        counts.append(year)
                                        #print(counts)
                                        df.loc[len(df)] = counts
                                        counts = []
                        else:
                            # silly formatting for 2013 deer and elk general in other years
                            table = gmu.next_element.next_element
                            #print(table)
                            body = table.find("tbody")
                            #print(body)
                            rows = body.find_all("tr")
                            #print(rows)
                            for row in rows[1:]:
                                #print(row)
                                data = row.find_all("td")
                                counter = 0
                                for point in data:
                                    if counter == 0:
                                        weapon = point.string
                                        #print(weapon)
                                    else:
                                        counts.append(point.string)
                                    counter += 1
                                counts.append(name)
                                counts.append(weapon)
                                counts.append(year)
                                df.loc[len(df)] = counts
                                counts = []
        index_names = df[ df['Weapon'] == "Totals"].index
        df.drop(index_names, inplace = True)
        #print(df)
    if hunttype=="special":
        if species == "deer":
            df = pd.DataFrame(columns=shortcols)
        else:
            df = pd.DataFrame(columns=elk_shortcols)
        # grab the data from the special hunts
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/"+species+"-special")
        page_html = response.text
        soup = BeautifulSoup(page_html, 'lxml')
        counts = []
        districts = soup.find_all(class_="accordion-content")
        for district in districts:
            body = district.find("tbody")
            rows = body.find_all("tr")
            olddata = body.find_all("th")
            if olddata:
                for row in rows:
                    gmu = row.find("th").string
                    gmu = gmu[0:3]
                    data = row.find_all("td")
                    counts.append(gmu)
                    for point in data:
                        counts.append(point.string)
                    
                    counts.append(year)
                    #print(counts)
                    df.loc[len(df)] = counts
                    counts = []
            else:
                # newer table format, 2018 and beyond
                for row in rows:
                    data = row.find_all(style="text-align: right;")
                    gmu = row.find(style="text-align: left;").string
                    gmu = gmu[0:3]
                    counts.append(gmu)
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    df.loc[len(df)] = counts
                    counts = []
    #else:
        # hunt is individual permit
        
    return df

In [ ]:
def bear(year: str, spring: bool) -> pd.DataFrame:
    """
    bear is a function that scrapes the WDFW bear hunting statistics web pages
    to extract information about the number of animals killed given a year and 
    type of hunt (general season vs. spring hunt) by location. Not all years had spring hunts but all year have 
    general (fall season) hunts.

    Args:
        year: a string with a four digit year between 2013 and 2023
        spring:  Boolean, true if spring hunt statistics desired, false is general season statistics desired
        

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    
    
    if not spring:
        df = pd.DataFrame(columns=bearcols)
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/black-bear-general")
        page_html = response.text
        
    else:
        df = pd.DataFrame(columns=bearspr)
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/black-bear-spring")
        page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    districts = soup.find_all(class_="accordion-content")
    for district in districts:
        if not spring:
            # general hunts
            body = district.find("tbody")
            rows = body.find_all("tr")
            for row in rows:
                # old table format
                data = row.find_all(align="right")
                if data:
                    gmu = row.find(align="left").string
                    gmu = gmu[0:3]
                    counts.append(gmu)
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    df.loc[len(df)] = counts
                    counts = []
                else:
                    # new table format
                    data = row.find_all(style="text-align: right;")
                    gmu = row.find(style="text-align: left;white-space: nowrap").string
                    gmu = gmu[0:3]
                    counts.append(gmu)
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    df.loc[len(df)] = counts
                    counts = []
        else:
            # spring bear hunts
            body = district.find("tbody")
            rows = body.find_all("tr")
            #data = rows[1].find_all(align="center")
            if len(rows) > 1:
                data = rows[1].find_all(align="center")
                # old table format
                for point in data[1:]:
                    counts.append(point.string)
                data = rows[3].find_all(align="center")
                for point in data:
                       counts.append(point.string)
                counts.append(year)
                df.loc[len(df)] = counts
                counts = []    
            else:
                for row in rows:
                # new table format
                    #print(row)
                    gmu = row.find(style="text-align: left;").next_element.next_element.next_element.string
                    gmu = gmu[4:]
                    counts.append(gmu)
                    data = row.find_all(style="text-align: right;")
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    df.loc[len(df)] = counts
                    counts = []    
    return df

In [ ]:
def turkey(year: str) -> pd.DataFrame:
    """
    turkey is a function that scrapes the WDFW turkey hunting statistics web pages
    to extract information about the number of animals killed given a year by location.

    Args:
        year: a string with a four digit year between 2013 and 2023
        
        

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    df = pd.DataFrame(columns=turkeycols)
    response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/turkey-statewide")
    page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    district = soup.find(class_="scroll")
    #print(district)
    body = district.find("tbody")
    #print(body)
    rows = body.find_all("tr")
    for row in rows:
        data = row.find_all("td")
        if len(data)>2:
            for point in data:              
                counts.append(point.string)
            counts.append(year)
            #print(counts)
            df.loc[len(df)] = counts
            counts = []
    return df

In [ ]:
def smallgame(year: str) -> pd.DataFrame:
     """
    smallgame is a function that scrapes the WDFW small game hunting statistics web pages
    to extract information about the number and type of animals killed given a year by location.

    Args:
        year: a string with a four digit year between 2013 and 2023

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    df = pd.DataFrame(columns=smallgamecols)
    response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/small-game")
    page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    districts = soup.find_all(class_="accordion-item")
    for district in districts:
        
        species = district.find("h2")
        name = species.contents[0].string
        #print(name)
        body = district.find("tbody")
        rows = body.find_all("tr")
            
        for row in rows:
            counts.append(name)
            data = row.find_all("td")
           
            if len(data) > 4 :
                # old table format
                for index, point in enumerate(data):
                    if index in [0,1,4,7]:
                        counts.append(point.string)
                counts.append(year)
                
            else:
                # new table format
                for point in data:
                    counts.append(point.string)
                counts.append(year)
                
            #print(counts)
            df.loc[len(df)] = counts
            counts = []
                
    return df

In [ ]:
def trapping(year: str) -> pd.DataFrame:
    """
    trapping is a function that scrapes the WDFW trapping statistics web pages
    to extract information about the number and type of animals killed given a year by location.

    Args:
        year: a string with a four digit year between 2013 and 2023

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    df = pd.DataFrame(columns=trappingcols)
    response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/trapping")
    page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    districts = soup.find_all(class_="accordion-content")
    for district in districts:
        #print(district)
        body = district.find("tbody")
        rows = body.find_all("tr")
        for row in rows:
            data = row.find_all("td")
            for point in data:
                counts.append(point.string)
            counts.append(year)
            #print(counts)
            df.loc[len(df)] = counts
            counts = []
    return df

In [ ]:
def biggame(species,year):
    """
    biggame is a function that scrapes the WDFW hunting statistics web pages
    to extract information about the number and type of large animals killed given a year by location.
    Includes moose, bighorn sheep, and mountain goats.

    Args:
        species: a string, one of "moose", "bighorn", or anything else (mountain goats)
        year: a string with a four digit year between 2013 and 2023

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    if species == "bighorn":
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/bighorn-sheep")
        df = pd.DataFrame(columns=bighorn_cols)
    elif species == "moose":
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/moose")
        df = pd.DataFrame(columns=moosecols)
    else:
        response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/mountain-goat")
        df = pd.DataFrame(columns=mountaingoat_cols)
    page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    districts = soup.find_all(class_="field-items")
    if len(districts) > 2:
        for district in districts[1:]:
            units = district.find_all("h3")
            if units:
                for unit in units:
                    table = unit.next_element.next_element.next_element
                    body = table.find("tbody")
                    rows = body.find_all("tr")
                    #print(rows)
                    for row in rows:
                        counts.append(unit.string[0:4])
                        data = row.find_all("td")
                        if species == "bighorn":
                            for index, point in enumerate(data):
                                if index in [0,1,2,3,4,7,8,9,10]:
                                    counts.append(point.string)
                            counts.append(year)
                        elif species == "mountaingoat":
                            for index, point in enumerate(data):
                                if index in [0,1,2,3,4,5,6,9,10,11,12]:
                                    counts.append(point.string)
                            counts.append(year)
                        else:
                            for point in data:
                                counts.append(point.string)
                            counts.append(year)
                        #print(counts)
                        df.loc[len(df)] = counts
                        counts = []
    else:
        # newer table format
        districts = soup.find_all(class_="block-region-content")
        for district in districts:
            tables = district.find_all("table")
            for table in tables:
                unit = table.next_element.next_element
                body = table.find("tbody")
                rows = body.find_all("tr")
                for row in rows:
                    counts.append(unit.string[0:4])
                    data = row.find_all(style="text-align: right;")
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    #print(counts)
                    df.loc[len(df)] = counts
                    counts = []
    return df

In [ ]:
def cougar(year: str) -> pd.DataFrame:
    """
    cougar is a function that scrapes the WDFW cougar hunting statistics web pages
    to extract information about the number and type of animals killed given a year by location.

    Args:
        year: a string with a four digit year between 2013 and 2023

    Returns:
        a Pandas dataframe with scraped information about hunt statistics

    """
    df = pd.DataFrame(columns=cougarcols)
    response = requests.get("https://wdfw.wa.gov/hunting/management/game-harvest/"+year+"/cougar")
    page_html = response.text
    soup = BeautifulSoup(page_html, 'lxml')    
    counts = []
    if year in ["2015","2016","2017"]:
        
        districts = soup.find_all(class_="accordion-content")
        for district in districts:
            #print(district)
            body = district.find_all("tbody")
            for table in body:
                rows = table.find_all("tr")
                for row in rows:
                    unit = row.find("th")
                    if unit:
                        gmu = unit.string
                        gmu = gmu[4:7]
                    else:
                        gmu = table.find(style="text-align: left;white-space: nowrap").string
                        gmu = gmu[0:3]
                    counts.append(gmu)
                    data = row.find_all("td")
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    #print(counts)
                    df.loc[len(df)] = counts
                    counts = []
    else:
        district = soup.find(class_="accordion-content")
        body = district.find_all("tbody")
        for table in body:
                rows = table.find_all("tr")
                for row in rows:
                    data = row.find_all("td")
                    for point in data:
                        counts.append(point.string)
                    counts.append(year)
                    #print(counts)
                    counts[0] = counts[0][:3]
                    df.loc[len(df)] = counts
                    counts = []
    return df

In [ ]:
def cleanup(df: pd.DataFrame) -> pd.DataFrame:
    df = df.replace({"n/a": None}, regex=True)
    return df

In [ ]:
def biggame_convert(species: str,df: pd.DataFrame) -> pd.DataFrame:
    """
    biggame_convert is a function that converts the unit numbers scraped from the WDFW website into game units
    found in WDFW historical hunting regulations.  This is used to accurately map the number of animals killed 
    in a given year.  
    
    Args:
        species: a string of either "moose", "bighorn", or anything else (mountain goats)
        df: a dataframe containing hunt statistics listed by game unit from the WDFW website

    Returns:
        a Pandas dataframe with converted game management units used for Tableau mapping

    """
    if species == "bighorn":
        LUT = bighornLUT
    elif species == "moose":
        LUT = mooseLUT
    else:
        LUT = MGLUT
    newunits = pd.merge(
        left=df,
        right=LUT,
        how='left',
        left_on=["Unit","Year"],
        right_on=["Unit","Year"]
    )
    newunits
    return newunits


In [ ]:
gmu_df = pd.DataFrame(gmu)
deer_gen_df = pd.DataFrame(columns=columns)
elk_gen_df = pd.DataFrame(columns=elkcols)
deer_spec_df = pd.DataFrame(columns=shortcols)
elk_spec_df = pd.DataFrame(columns = elk_shortcols)
bear_df = pd.DataFrame(columns=bearcols)
bear_spr_df = pd.DataFrame(columns=bearspr)
turkey_df = pd.DataFrame(columns=turkeycols)
smallgame_df = pd.DataFrame(columns=smallgamecols)
trapping_df = pd.DataFrame(columns=trappingcols)
moose_df = pd.DataFrame(columns=moosecols)
bighorn_df = pd.DataFrame(columns=bighorn_cols)
mountaingoat_df = pd.DataFrame(columns=mountaingoat_cols)
cougar_df = pd.DataFrame(columns=cougarcols)

for year in year_list:
    print(year)
    if year in ["2013","2014"]:
        # no turkey, spring bear, no cougar
        print("Adding spring bear records")
        beardf = bear(year,True)
        bear_spr_df = pd.concat([bear_spr_df,beardf],ignore_index=True)

    if year in ["2015","2016","2017"]:
         # no turkey, spring bear, cougar
        print("Adding spring bear records")
        beardf = bear(year,True)
        bear_spr_df = pd.concat([bear_spr_df,beardf],ignore_index=True)
        print("Adding cougar records")
        cougardf = cougar(year)
        cougar_df = pd.concat([cougar_df,cougardf],ignore_index=True)
     
    if year in ["2018","2019","2020","2021"]:
        # turkey, spring bear, cougar
        print("Adding spring bear records")
        beardf = bear(year,True)
        bear_spr_df = pd.concat([bear_spr_df,beardf],ignore_index=True)
        print("Adding turkey records")
        turkeydf = turkey(year)
        turkey_df = pd.concat([turkey_df,turkeydf],ignore_index=True)
        print("Adding cougar records")
        cougardf = cougar(year)
        cougar_df = pd.concat([cougar_df,cougardf],ignore_index=True)
     
    if year in ["2022"]:
        # turkey, no spring bear, cougar
        print("Adding turkey records")
        turkeydf = turkey(year)
        turkey_df = pd.concat([turkey_df,turkeydf],ignore_index=True)
        print("Adding cougar records")
        cougardf = cougar(year)
        cougar_df = pd.concat([cougar_df,cougardf],ignore_index=True)
    
    if year in ["2023"]:
        # turkey, no spring bear, no cougar
        print("Adding turkey records")
        turkeydf = turkey(year)
        turkey_df = pd.concat([turkey_df,turkeydf],ignore_index=True)

    # every year has deer general+special, elk general+special, fall bear, trapping, small game, big game (bighorn, moose, mountain goat)
    print("Adding deer general season records")
    deerdf = deerelk("deer","general",year) 
    deer_gen_df = pd.concat([deer_gen_df,deerdf],ignore_index=True)
    print("Adding deer special season records")
    deefdf = deerelk("deer","special",year)
    deer_spec_df = pd.concat([deer_spec_df,deerdf],ignore_index=True)
    print("Adding elk general season records")
    elkdf = deerelk("elk","general",year)
    elk_gen_df = pd.concat([elk_gen_df,elkdf],ignore_index=True)
    print("Adding elk special season records")
    elkdf = deerelk("elk","special",year)
    elk_spec_df = pd.concat([elk_spec_df,elkdf],ignore_index=True)
    print("Adding fall bear records")
    beardf = bear(year,False)
    bear_df = pd.concat([bear_df,beardf],ignore_index=True)
    print("Adding small game records")
    smallgamedf = smallgame(year)
    smallgame_df = pd.concat([smallgame_df,smallgamedf],ignore_index=True)
    print("Adding trapping records")
    trappingdf = trapping(year)
    trapping_df = pd.concat([trapping_df,trappingdf],ignore_index=True)
    print("Adding bighorn sheep records")
    bighorndf = biggame("bighorn",year)
    bighorn_df = pd.concat([bighorn_df,bighorndf],ignore_index=True)
    print("Adding moose records")
    moosedf = biggame("moose",year)
    moose_df = pd.concat([moose_df,moosedf],ignore_index=True)
    print("Adding mountain goat records")
    mountaingoatdf = biggame("mountaingoat",year)
    mountaingoat_df = pd.concat([mountaingoat_df,mountaingoatdf],ignore_index=True)

In [ ]:
# change n/a to None where it appears in dataframes
deer_gen_df = cleanup(deer_gen_df)
deer_spec_df = cleanup(deer_spec_df)
elk_gen_df = cleanup(elk_gen_df)


In [ ]:
# convert data types of big game for merging map unit columns from lookup tables
bighorn_df["Unit"] = bighorn_df["Unit"].astype(int)
bighorn_df["Year"] = bighorn_df["Year"].astype(int)
moose_df["Unit"] = moose_df["Unit"].astype(int)
moose_df["Year"] = moose_df["Year"].astype(int)
mountaingoat_df["Unit"] = mountaingoat_df["Unit"].astype(int)
mountaingoat_df["Year"] = mountaingoat_df["Year"].astype(int)
# merge the lookup tables with the big game dataframes to provide map unit columns for Tableau maps
bighorn_merge_df = biggame_convert("bighorn",bighorn_df)
bighorn_merge_df["MapUnit"] = bighorn_merge_df["MapUnit"].astype(int)
moose_merge_df = biggame_convert("moose",moose_df)
moose_merge_df["MapUnit"] = moose_merge_df["MapUnit"].astype(int)
mountaingoat_merge_df = biggame_convert("mountaingoat",mountaingoat_df)
mountaingoat_merge_df["MapUnit"] = mountaingoat_merge_df["MapUnit"].astype(int)

In [ ]:
with pd.ExcelWriter('wahunt.xlsx') as writer:  

    gmu_df.to_excel(writer,sheet_name="GMU")
    
    deer_gen_df.to_excel(writer, sheet_name='Deer General')

    deer_spec_df.to_excel(writer, sheet_name='Deer Special')

    elk_gen_df.to_excel(writer, sheet_name='Elk General')

    elk_spec_df.to_excel(writer, sheet_name='Elk Special')

    bear_df.to_excel(writer, sheet_name='Bear')

    turkey_df.to_excel(writer, sheet_name='Turkey')

    cougar_df.to_excel(writer, sheet_name='Cougar')

    smallgame_df.to_excel(writer, sheet_name='Small Game')

    trapping_df.to_excel(writer, sheet_name='Trapping')

    bighorn_merge_df.to_excel(writer, sheet_name='Bighorn Sheep')

    moose_merge_df.to_excel(writer, sheet_name='Moose')

    mountaingoat_merge_df.to_excel(writer, sheet_name='Mountain Goat')